In [2]:
import pandas as pd
import gc
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Union


import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px






"""   
This note book assumes that we have already got the benchmark job data and data from promethues. 
It then goes through jobs, one by one, and by getting the node and starting and endtime it gets ..
their corrosponding trace from the promehues zip files.
"""


'   \nThis note book assumes that we have already got the benchmark job data and data from promethues. \nIt then goes through jobs, one by one, and by getting the node and starting and endtime it gets ..\ntheir corrosponding trace from the promehues zip files.\n'

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

folder_path =  Path('/projects/2/prjs1098/system_analytics_2024/prom_data')
prom_files = ['metrics_stats_9.parquet.gzip', 'metrics_stats_6.parquet.gzip']
benchmark_file= folder_path / 'benchmark_single_node_job_data' /'benchmark_job_data_cleaned.csv'


In [4]:
df_benchmark = pd.read_csv(benchmark_file)
df_benchmark.set_index('job_id', inplace=True)
df_benchmark.head()

,submit,eligible,start,end,elapsed,slurm_job_id,state,alloccpus,totalcpu,node_list,name,system,result,fail_reason,fail_phase,environment,time_compile,time_performance,time_run,time_sanity,time_setup,time_total,pref_name,pref_reference,pref_thres_lower,pref_thres_upper,pref_unit,pref_value,valid_prog_environs,valid_systems,executable,executable_opts,num_tasks,num_tasks_per_node,num_cpus_per_task,time_limit,prerun_cmds,postrun_cmds,modules,env_vars,readonly_files
job_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8616980,2024-11-20T11:54:35,2024-11-20T11:54:35,2024-11-20T11:54:52,2024-11-20T12:07:43,00:12:51,8616980,COMPLETED,72,03:29:30,gcn49,GROMACS_GPU %n_gpus=1,snellius:gpu_a100,success,NaN,NaN,eb-foss,0.029076,0.227419,792.553260,0.096275,0.527797,794.377215,perf,0,NaN,NaN,ns/day,16.529,['eb-foss'],"['snellius:gpu_a100', 'snellius:gpu_h100']",gmx_mpi,['mdrun -v -pin on -dlb yes -resethway -deffnm...,1,1.0,18.0,1800.0,['cp /gpfs/work4/1/reframe0/resourcesdir/groma...,[],['gromacs-foss-gpu'],{},[]
8616982,2024-11-20T11:54:38,2024-11-20T11:54:38,2024-11-20T11:54:41,2024-11-20T12:07:18,00:12:37,8616982,COMPLETED,64,02:57:44,gcn149,GROMACS_GPU %n_gpus=1,snellius:gpu_h100,success,NaN,NaN,eb-foss,0.028724,0.073034,766.922158,0.097536,0.048740,770.778406,perf,0,NaN,NaN,ns/day,25.705,['eb-foss'],"['snellius:gpu_a100', 'snellius:gpu_h100']",gmx_mpi,['mdrun -v -pin on -dlb yes -resethway -deffnm...,1,1.0,16.0,1800.0,['cp /gpfs/work4/1/reframe0/resourcesdir/groma...,[],['gromacs-foss-gpu'],{},[]
8616984,2024-11-20T11:54:40,2024-11-20T11:54:40,2024-11-20T11:54:52,2024-11-20T12:08:01,00:13:09,8616984,COMPLETED,128,06:26:11,tcn109,GROMACS_CPU %n_cores=32,snellius:rome,success,NaN,NaN,eb-foss-mpi,0.028928,0.179587,806.839507,0.187784,0.047647,813.222719,perf,0,NaN,NaN,ns/day,3.262,['eb-foss-mpi'],"['snellius:rome', 'snellius:genoa']",gmx_mpi,['mdrun -v -pin on -dlb yes -resethway -deffnm...,32,32.0,1.0,1800.0,['cp /gpfs/work4/1/reframe0/resourcesdir/groma...,[],['gromacs-foss'],{},[]
8616986,2024-11-20T11:54:43,2024-11-20T11:54:43,2024-11-20T11:54:52,2024-11-20T12:07:56,00:13:04,8616986,COMPLETED,192,06:29:00,tcn575,GROMACS_CPU %n_cores=32,snellius:genoa,success,NaN,NaN,eb-foss-mpi,0.029197,0.046268,798.165663,0.052712,0.047641,806.888496,perf,0,NaN,NaN,ns/day,3.343,['eb-foss-mpi'],"['snellius:rome', 'snellius:genoa']",gmx_mpi,['mdrun -v -pin on -dlb yes -resethway -deffnm...,32,32.0,1.0,1800.0,['cp /gpfs/work4/1/reframe0/resourcesdir/groma...,[],['gromacs-foss'],{},[]
8617011,2024-11-20T11:57:08,2024-11-20T11:57:08,2024-11-20T11:57:22,2024-11-20T12:02:20,00:04:58,8617011,COMPLETED,128,08:15:03,tcn111,HPCG_MPIOnly %nnodes=1,snellius:rome,success,NaN,NaN,eb-foss-mpi,120.110956,0.027074,318.363357,0.167167,0.051292,472.052970,gflops,0,NaN,NaN,Gflop/s,37.848,"['eb-foss-mpi', 'eb-intel-mpi']","['lisa:silver', 'lisa:gold', 'snellius:rome', ...",hpcg/bin/xhpcg,"['--nx=104', '--ny=104', '--nz=104', '-t2']",128,128.0,1.0,3600.0,[],[],[],{},[]


In [5]:

def get_trace_for_node_time(df: pd.DataFrame, node_name: list[str], 
                            start_time: Union[int, str], end_time: Union[int, str]) -> dict[str, pd.DataFrame]:
    df_dict = {}
    if not isinstance(start_time, int):
        start_time = pd.Timestamp(start_time).timestamp()  # put -15 if you want to shift the start time
    if not isinstance(end_time, int):
        end_time = pd.Timestamp(end_time).timestamp()  # put +15 to shift the end time
    # print(start_time, end_time)

    try:
        mask = ((df['node']==node_name) & (df['timestamp']<=end_time)
                                    & (start_time<=df['timestamp']))
        df_node = df[mask].copy()
        if len(df_node)==0:
            raise KeyError(f"No data found for node '{node_name}' within the specified time range.")
        else:
            # df_node.drop_duplicates(inplace=True)
            df_dict[node_name] = df_node
    except KeyError as e:
        print(e)
        df_dict[node_name] = pd.DataFrame({})
    except:
        df_dict[node_name] = pd.DataFrame({})
        
    return df_dict, start_time, end_time



def get_trace_for_benchmark(df_benchmark, df_prom):
      job_data_prom_list = []
      for job_id, row in df_benchmark.iterrows():
            start_time = row['start']
            end_time = row['end']
            node_list = row['node_list']
            print(f"Benchamrk job_id: {job_id}")
            print(f"Benchmark nodes and times: {node_list}, {start_time}, {end_time}")
            df_dict, start_time_sec, end_time_sec = get_trace_for_node_time(df_prom, node_list, start_time, end_time)
            for key, value  in df_dict.items():
                  print("Prometheus data:")
                  try:
                        print(f"{key}:", pd.to_datetime(value['timestamp'].min(), unit='s'),
                              pd.to_datetime(value['timestamp'].max(),unit='s'))
                        value['job_id'] = job_id
                        job_data_prom_list.append(value)
                  except KeyError:
                        print("There was no data, please pass the correct data frame to the function")
      return job_data_prom_list

In [ ]:
job_data_prom_list = []
for prom_file in prom_files:
    df_prom = pd.read_parquet(folder_path / prom_file)
    job_data_prom_list += get_trace_for_benchmark(df_benchmark, df_prom)

Benchamrk job_id: 8616980
Benchmark nodes and times: gcn49, 2024-11-20T11:54:52, 2024-11-20T12:07:43
"No data found for node 'gcn49' within the specified time range."
Prometheus data:
There was no data, please pass the correct data frame to the function
Benchamrk job_id: 8616982
Benchmark nodes and times: gcn149, 2024-11-20T11:54:41, 2024-11-20T12:07:18
"No data found for node 'gcn149' within the specified time range."
Prometheus data:
There was no data, please pass the correct data frame to the function
Benchamrk job_id: 8616984
Benchmark nodes and times: tcn109, 2024-11-20T11:54:52, 2024-11-20T12:08:01
"No data found for node 'tcn109' within the specified time range."
Prometheus data:
There was no data, please pass the correct data frame to the function
Benchamrk job_id: 8616986
Benchmark nodes and times: tcn575, 2024-11-20T11:54:52, 2024-11-20T12:07:56
"No data found for node 'tcn575' within the specified time range."
Prometheus data:
There was no data, please pass the correct data 

Prometheus data:
gcn131: 2024-12-11 09:14:15 2024-12-11 09:26:15
Benchamrk job_id: 8995291
Benchmark nodes and times: tcn356, 2024-12-11T09:14:16, 2024-12-11T09:26:51
Prometheus data:
tcn356: 2024-12-11 09:14:30 2024-12-11 09:26:45
Benchamrk job_id: 8995292
Benchmark nodes and times: tcn561, 2024-12-11T09:14:15, 2024-12-11T09:26:57
Prometheus data:
There was no data, please pass the correct data frame to the function
Benchamrk job_id: 8995293
Benchmark nodes and times: tcn354, 2024-12-11T09:14:16, 2024-12-11T09:18:28


In [ ]:

# sort based on job id and timestamp
df_benchmark_prom = pd.concat(job_data_prom_list, axis=0)
# df_benchmark_prom = pd.merge(df_benchmark, df_job_id_prom, on='job_id')
df_benchmark_prom.sort_values(['job_id', 'timestamp'], inplace=True)
# create a plot based on 4-6 signals
df_benchmark_prom.head()

In [ ]:
df_benchmark_prom.set_index('job_id', inplace=True)
min_time_per_job = df_benchmark_prom.groupby(by=df_benchmark_prom.index)['timestamp'].min()
df_benchmark_prom['shifted_time'] = df_benchmark_prom['timestamp'] - min_time_per_job
df_benchmark_prom.reset_index(inplace=True)


In [ ]:

# here is a good place to save the data and go to the analysis section
# df_benchmark_prom.to_parquet(folder_path / 'benchmark_single_node_job_data' /'benchmark_job_data_with_prom.parquet.gzip',
#                              compression='gzip')